In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
img_id = "CA184632V0333"
calc_types = ["sat", "img"]
overwrite = False

output_fld = "/data/ATM/data_1/georef"

In [3]:
# load the image
import src.load.load_image as li
image = li.load_image(img_id)

In [4]:
import src.base.connect_to_database as ctd

sql_string = f"SELECT * FROM images WHERE image_id='{img_id}'"
conn = ctd.establish_connection()
data = ctd.execute_sql(sql_string, conn)

month = data.iloc[0]['date_month']


<h4>Load the shapefiles with the georeferenced images</h4>

In [5]:
import src.display.display_shapes as ds

import src.load.load_shape_data as lsd

path_sat_shapefile = "/data/ATM/data_1/georef/sat.shp"

# load the shapefile with the images
georef_shape_data = lsd.load_shape_data(path_sat_shapefile)

# get the geometry
georef_shapes = georef_shape_data.geometry

# get the unique ids from the shapes
georef_ids = georef_shape_data['image_id'].tolist()

In [6]:
# Filter out geo-referenced images and footprints that don't match the flight path
filtered_georef_ids = []
filtered_georef_shapes = []
for i, geo_id in enumerate(georef_ids):
    if geo_id[2:6] == img_id[2:6]:
        filtered_georef_ids.append(geo_id)
        filtered_georef_shapes.append(georef_shapes[i])

# Replace the original lists with the filtered lists
georef_ids = filtered_georef_ids
georef_shapes = filtered_georef_shapes

In [9]:
import src.georef.georef_calc as gc

GCalc = gc.GeorefCalc()

transform, residuals, tps, conf = GCalc.georeference(image, img_id, georef_ids, georef_shapes)

<h4>Verify the image geometry and position</h4>

In [8]:
import src.georef.snippets.verify_image_geometry as vig
import src.georef.snippets.verify_image_position as vip

# check if the geometry of the image is valid
valid_geometry, reason = vig.verify_image_geometry(image, transform)

# check if the image fits in the position of the other images
#valid_position, reason = vip.verify_image_position(image, transform, georef_shapes)

print(valid_geometry)


True


<h4>Save the data</h4>

In [43]:
import numpy as np
import pandas as pd

import src.georef.snippets.apply_transform as af
import src.georef.snippets.convert_image_to_footprint as citf

import src.export.export_geometry as eg

# merge tps and conf
tps_conf = np.concatenate([tps, conf.reshape(-1, 1)], axis=1)

# define path to shape file
path_shp_file = f"{output_fld}/calc.shp"

# apply the transform and save geo-referenced tiff
af.apply_transform(image, transform, f"{output_fld}/calc/{img_id}.tif")

# save transform and points
np.savetxt(f"{output_fld}/calc/{img_id}_transform.txt", transform, fmt='%.5f')
np.savetxt(f"{output_fld}/calc/{img_id}_points.txt", tps_conf, fmt=['%i', '%i', '%.2f', '%.2f', '%.3f'])

# create a footprint for this image
footprint = citf.convert_image_to_footprint(image, transform)

# define attributes
attributes = {
    'image_id': img_id,
    'month': month,
    'num_tps': tps.shape[0],
    'avg_conf': round(np.mean(conf), 3),
    'avg_resi': round(np.mean(residuals), 3),
}

attributes = pd.DataFrame.from_dict(attributes, orient='index').T

# save footprint to shp file
eg.export_geometry(footprint, path_shp_file,
                   attributes=attributes, key_field="image_id",
                   overwrite_file=False,
                   overwrite_entry=True, attach=True)

print(f"{img_id} successfully saved")


CA184632V0333 successfully saved
